In [1]:
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings

In [2]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [3]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(docs, embeddings)

c:\Users\MASTER\Desktop\langchain_models\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
base_retriever = vector_store.as_retriever(search_kwargs={"k": 5})

In [5]:
llm = ChatOpenAI(
   model="gpt-oss-120b",    
    base_url="https://api.cerebras.ai/v1",   # Cerebras endpoint
)
compressor = LLMChainExtractor.from_llm(llm)

In [6]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)


In [7]:
query = "What is Photosynthesis?"
compression_results = compression_retriever.invoke(query)

In [8]:
for i , doc in enumerate(compression_results):
    print(f"Result {i+1}:")
    print(doc.page_content)
    print(f"Source: {doc.metadata['source']}")
    print("-" * 50)

Result 1:
Photosynthesis is the process by which green plants convert sunlight into energy.
Source: Doc1
--------------------------------------------------
Result 2:
The chlorophyll in plant cells captures sunlight during photosynthesis.
Source: Doc2
--------------------------------------------------
Result 3:
Photosynthesis does not occur in animal cells.
Source: Doc4
--------------------------------------------------
